In [1]:
import pandas as pd
import numpy as np
import os
import nltools as nlt
import nilearn as nil
import nibabel as nib
import warnings
import glob
import random
import pickle
from operator import itemgetter
import datetime

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


This file loads the behavioral data from the output file, matches it to the list of betas we have, and then splits it into train and test groups.

As long as input data doesn't change it shouldn't change which subjects are allocated to each group each time.

If input data does change, then you can use the previous output of this file to manually hardcode which groups already-classified subjects should be in using `already_peaked_paths` files: see below.

In [48]:
ml_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

In [2]:
#load behavioral data

import dev_wtp_io_utils
behavioral_data_path = '/gpfs/projects/sanlab/shared/DEV/nonbids_data/Tasks/WTP/output/'
[run_list, run_event_df_list, run_event_stage_df_list] = dev_wtp_io_utils.get_comprehensive_run_data_from_mat_dir(behavioral_data_path)



subject 040, subject 039, subject 168, subject 063, subject 224, subject 129, subject 067, subject 150, subject 159, 
SUBJ 159, 2, run14
no data for this run.
subject 004, subject 091, subject 180, subject 008, subject 095, subject 099, subject 032, 
SUBJ 032, 2, run2
no data for this run.
subject 218, subject 036, subject 171, subject 100, 
SUBJ 100, 3, run2
no data for this run.
subject 109, subject 041, subject 060, subject 059, subject 130, subject 223, subject 203, subject 064, subject 068, subject 001, subject 087, subject 192, subject 005, subject 121, subject 999, 
SUBJ 999, 2, run1
no data for this run.


SUBJ 999, 2, run2
Data for this run is not in the expected format or is missing. Skipping this run.
time: 29-Oct-2020 17:09:09

StartTime: 3035.874545661

Jitter: array of length 16

TrialStart: array of length 2

ISI: array of length 2

FoodOn: array of length 2

BidOn: array of length 2

FoodOnset: array of length 2

BidOnset: array of length 2

FoodDuration: array of lengt

This file is a snapshot from: https://docs.google.com/spreadsheets/d/1NywNwiRxwlAOjb2eZ7q58MklRWUGvgE5hlaILurwFIs/edit#gid=0

In [3]:
#load design data from the official list of task design.
#this is NOT per subject.
betas = pd.read_csv("/gpfs/projects/sanlab/bsmith16/data/DEV QC and Exclusions - betas_machine_readable.csv")
wtp_betas = betas[betas.task=='WTP betas']
#get the betas just for the four runs
wtp_betas_runfiles = wtp_betas[[bt in ['run1','run2','run3','run4'] for bt in wtp_betas.type]].copy()
#number them
wtp_betas_runfiles['event_id'] = wtp_betas_runfiles.groupby('type').cumcount()+1

In [4]:
wtpw1_behavdesign = wtp_betas_runfiles.merge(run_event_df_list, how='left',
                         left_on=['type','event_id'],
                        right_on=['run','event_id'])

wtpw1_behavdesign = wtpw1_behavdesign[(wtpw1_behavdesign.wave==1)]

wtpw1_behavdesign.sample(5)


,beta,type,task,event_id,isi_pre,onset,duration,food_pic,food_num,cond,health_cond,liking_cond,liking_rating,response,isi_post,end,run,wave,subject
29863,beta_0066.nii,run4,WTP betas,3,1.780283,30.029739,6.531829,Pistachios.bmp,3,healthy_liked,healthy,liked,4,8,0.095117,36.561569,run4,1,DEV099
724,beta_0002.nii,run1,WTP betas,2,3.000000,18.697115,6.531634,Kiwi.bmp,2,healthy_liked,healthy,liked,4,5,1.780283,25.228749,run1,1,DEV040
14234,beta_0025.nii,run2,WTP betas,4,0.095117,39.673193,6.526455,3Musketeers.bmp,4,unhealthy_disliked,unhealthy,disliked,1,5,2.680966,46.199648,run2,1,DEV155
25727,beta_0050.nii,run3,WTP betas,8,2.083661,84.857648,6.525055,HotTamales.bmp,8,unhealthy_disliked,unhealthy,disliked,1,5,1.584060,91.382703,run3,1,DEV216
29027,beta_0058.nii,run3,WTP betas,16,1.421652,168.656373,6.515888,peachrings.bmp,16,unhealthy_liked,unhealthy,liked,1,5,NaN,175.172261,run3,1,DEV015


In [5]:
#iterate through subject directories.

subject_dirs_all = glob.glob('/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/betaseries/sub-DEV*')
subject_dirs_all.sort()

In [6]:
#make some labels based on this list
import re
sub_labels_all = [(re.search("DEV(\\d\\d\\d)",sid)[0]) for sid in subject_dirs_all]
sub_ids_all = [(int)(re.search("DEV(\\d\\d\\d)",sid)[1]) for sid in subject_dirs_all]

## Deal with incomplete data

1. Events listed in betas that do not have a betas file

2. Subjects listed in subject_dirs that don't have corresponding beta data

In [7]:
wtpw1_behavdesign

,beta,type,task,event_id,isi_pre,onset,duration,food_pic,food_num,cond,health_cond,liking_cond,liking_rating,response,isi_post,end,run,wave,subject
3,beta_0001.nii,run1,WTP betas,1,3.000000,6.080660,6.564707,SourPunch.bmp,1,unhealthy_liked,unhealthy,liked,4,NULL,3.0,12.645367,run1,1,DEV040
6,beta_0001.nii,run1,WTP betas,1,3.000000,6.068091,6.548297,M&Ms.bmp,1,unhealthy_liked,unhealthy,liked,4,6,3.0,12.616389,run1,1,DEV039
9,beta_0001.nii,run1,WTP betas,1,3.000000,6.076286,6.548571,MacNCheese.bmp,1,unhealthy_liked,unhealthy,liked,3,8,3.0,12.624857,run1,1,DEV168
13,beta_0001.nii,run1,WTP betas,1,3.000000,6.085906,6.553895,FamousAmos.bmp,1,unhealthy_liked,unhealthy,liked,2,6,3.0,12.639801,run1,1,DEV063
15,beta_0001.nii,run1,WTP betas,1,3.000000,6.064116,6.546106,M&Ms.bmp,1,unhealthy_liked,unhealthy,liked,3,7,3.0,12.610221,run1,1,DEV224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35190,beta_0079.nii,run4,WTP betas,16,1.421652,168.783176,6.532094,lunchables_turkey.bmp,16,unhealthy_liked,unhealthy,liked,1,NULL,NaN,175.315270,run4,1,DEV179
35191,beta_0079.nii,run4,WTP betas,16,1.421652,168.726133,6.532007,FamousAmos.bmp,16,unhealthy_liked,unhealthy,liked,3,6,NaN,175.258140,run4,1,DEV098
35194,beta_0079.nii,run4,WTP betas,16,1.421652,168.782857,6.533177,SourCreamChips.bmp,16,unhealthy_liked,unhealthy,liked,4,7,NaN,175.316034,run4,1,DEV108
35195,beta_0079.nii,run4,WTP betas,16,1.421652,168.644515,6.516474,Poptart1.bmp,16,unhealthy_liked,unhealthy,liked,2,5,NaN,175.160989,run4,1,DEV035


In [8]:
pd.set_option('display.max_rows', 50)

Exclude from consideration subjects that have too few events (i.e., less than 64).

In [9]:
events_per_subject = wtpw1_behavdesign.groupby('subject').duration.count()
subjects_with_too_few_events = events_per_subject[events_per_subject<64].index.tolist()

wtpw1_behavdesign_clean = wtpw1_behavdesign.loc[[(s in subjects_with_too_few_events)==False for s in wtpw1_behavdesign.subject]]


In [10]:
print(wtpw1_behavdesign_clean.shape, wtpw1_behavdesign.shape)

(13120, 19) (13376, 19)


The following subjects have behavioral data recorded in the MATLAB behavioral output folder via the function `get_comprehensive_run_data_from_mat_dir`, but they do not have a beta.nii folder:

In [33]:
subjs_sorted = wtpw1_behavdesign_clean.subject.unique()
subjs_sorted.sort()
subjs_sorted

array(['DEV001', 'DEV002', 'DEV004', 'DEV005', 'DEV006', 'DEV007',
       'DEV008', 'DEV009', 'DEV010', 'DEV011', 'DEV012', 'DEV013',
       'DEV014', 'DEV015', 'DEV016', 'DEV017', 'DEV018', 'DEV019',
       'DEV020', 'DEV021', 'DEV022', 'DEV023', 'DEV024', 'DEV025',
       'DEV026', 'DEV027', 'DEV028', 'DEV029', 'DEV030', 'DEV031',
       'DEV032', 'DEV033', 'DEV034', 'DEV035', 'DEV036', 'DEV037',
       'DEV038', 'DEV039', 'DEV040', 'DEV041', 'DEV042', 'DEV043',
       'DEV044', 'DEV045', 'DEV046', 'DEV047', 'DEV048', 'DEV049',
       'DEV050', 'DEV051', 'DEV052', 'DEV053', 'DEV054', 'DEV055',
       'DEV056', 'DEV057', 'DEV058', 'DEV059', 'DEV060', 'DEV061',
       'DEV062', 'DEV063', 'DEV064', 'DEV065', 'DEV066', 'DEV067',
       'DEV068', 'DEV069', 'DEV070', 'DEV071', 'DEV073', 'DEV074',
       'DEV075', 'DEV076', 'DEV077', 'DEV078', 'DEV079', 'DEV080',
       'DEV081', 'DEV083', 'DEV084', 'DEV085', 'DEV086', 'DEV087',
       'DEV088', 'DEV089', 'DEV090', 'DEV091', 'DEV093', 'DEV0

In [11]:

subjects_with_behavioral_no_beta_folder = set(wtpw1_behavdesign_clean.subject).difference(set(sub_labels_all))
subjects_with_behavioral_no_beta_folder = list(subjects_with_behavioral_no_beta_folder)
subjects_with_behavioral_no_beta_folder.sort()
print( subjects_with_behavioral_no_beta_folder)

['DEV002', 'DEV007', 'DEV011', 'DEV020', 'DEV032', 'DEV047', 'DEV063', 'DEV064', 'DEV067', 'DEV075', 'DEV078', 'DEV088', 'DEV090', 'DEV093', 'DEV094', 'DEV095', 'DEV096', 'DEV097', 'DEV098', 'DEV099', 'DEV100', 'DEV101', 'DEV102', 'DEV103', 'DEV104', 'DEV105', 'DEV106', 'DEV107', 'DEV108', 'DEV109', 'DEV110', 'DEV111', 'DEV112', 'DEV113', 'DEV115', 'DEV116', 'DEV117', 'DEV118', 'DEV119', 'DEV120', 'DEV121', 'DEV122', 'DEV123', 'DEV124', 'DEV126', 'DEV127', 'DEV128', 'DEV129', 'DEV130', 'DEV131', 'DEV132', 'DEV133', 'DEV134', 'DEV135', 'DEV137', 'DEV138', 'DEV139', 'DEV140', 'DEV141', 'DEV143', 'DEV144', 'DEV145', 'DEV147', 'DEV149', 'DEV150', 'DEV151', 'DEV152', 'DEV153', 'DEV154', 'DEV155', 'DEV156', 'DEV157', 'DEV158', 'DEV159', 'DEV161', 'DEV163', 'DEV164', 'DEV166', 'DEV167', 'DEV168', 'DEV169', 'DEV170', 'DEV171', 'DEV173', 'DEV174', 'DEV176', 'DEV177', 'DEV178', 'DEV179', 'DEV180', 'DEV181', 'DEV182', 'DEV183', 'DEV185', 'DEV186', 'DEV187', 'DEV188', 'DEV189', 'DEV190', 'DEV191',

The following subjects have a `beta.nii` folder but they do not have behavioral data recorded in the MATLAB behavioral output folder via the function `get_comprehensive_run_data_from_mat_dir`.

See: https://docs.google.com/presentation/d/1K-nFrZYE6rR8t0myNyacB7frBzV3B1--nMqPhVkwL8E/edit#slide=id.gd9fcc4129a_0_0

In [12]:
behavioral_data_subjects = set(wtpw1_behavdesign_clean.subject)

In [13]:
subjects_with_beta_folder_no_behavioral = set(sub_labels_all).difference(behavioral_data_subjects)

In [14]:
subjects_with_beta_folder_no_behavioral

{'DEV072', 'DEV082'}

Finally we want to remove those subjects with no behavioral data from the list...

In [15]:
subject_dirs_usable = [d for d, sl in zip(subject_dirs_all,sub_labels_all) if sl not in subjects_with_beta_folder_no_behavioral]
sub_labels_usable = [sl for d, sl in zip(subject_dirs_all,sub_labels_all) if sl not in subjects_with_beta_folder_no_behavioral]

In [16]:
ts = datetime.datetime.now().strftime('%Y%m%dT%H%M%S')

In [19]:
added_labels = pd.DataFrame({"sub_label":sub_labels_usable})
added_labels['added'] = datetime.datetime.now()
added_labels.to_csv(ml_data_path + "useable_subject_labels.csv",index=False)

### Manage split and select subjects

In [20]:
# #make some labels based on this list
# import re
# sub_labels = [(re.search("DEV(\\d\\d\\d)",sid)[0]) for sid in subject_dirs]
# sub_ids = [(int)(re.search("DEV(\\d\\d\\d)",sid)[1]) for sid in subject_dirs]

Get the subjects we've already peaked at. These were _also_ chosen randomly, just via a prior process. They'll be part of the train group.

In [21]:
already_peaked_paths = pd.concat(
    [pd.read_csv("/gpfs/projects/sanlab/bsmith16/data/train_subjs_25_20210601T131602.csv"),
    pd.read_csv("/gpfs/projects/sanlab/bsmith16/data/holdout_subjs_5_20210601T131602.csv")]
)


already_peaked_labels = [(re.search("DEV(\\d\\d\\d)",sid)[0]) for sid in already_peaked_paths['0']]


In [22]:
pre_determined_training_subjs = already_peaked_labels

Determine the sizes of the groups we want to use:

In [23]:
total_sample_size = len(subject_dirs_usable)# - (len(subject_dirs_usable) % 5)
holdout_size = 5
holdout_n = (int)(total_sample_size/holdout_size)

In [24]:
holdout_n

15

In [25]:
print(total_sample_size,holdout_n,total_sample_size-holdout_n)

75 15 60


The only way to get a REALLY stable set of useable subjects, robust to even corrections to the dataset, is to make this list manually, so that's what we'll do.

In [26]:

ts = datetime.datetime.now().strftime('%Y%m%dT%H%M%S')

In [27]:
ts

'20211027T171101'

In [ ]:
test_train_df = pd.DataFrame({"sub_label":sub_labels_usable})

Now we get the pre-determined subjects marked as 'train' first

In [ ]:
test_train_df['SplitGroup'] = None

In [ ]:
for sl in test_train_df.sub_label:
    if sl in pre_determined_training_subjs:
        test_train_df.loc[test_train_df.sub_label==sl,'SplitGroup']='Train'
    

In [ ]:
test_train_df.SplitGroup.value_counts()

And _now_ we can use the order of the subjects in the randomization to select remaining subjects to allocate to the two groups. 

Set a random order for the subjects to be selected from...

In [ ]:
unallocated_subjects = test_train_df[test_train_df.SplitGroup.isnull()].sub_label.tolist()

In [ ]:
random.seed(a=1622149277.6487303)
subj_labels_randomized = random.sample(unallocated_subjects,k=len(unallocated_subjects)) # add an extra 5 as a hold-out

In [ ]:
holdout_subjs = subj_labels_randomized[0:holdout_n]


In [ ]:
for hs in holdout_subjs:
    test_train_df.loc[test_train_df.sub_label==hs,'SplitGroup']='Holdout'

In [ ]:
#mark the remaining subjects as for training
test_train_df.loc[test_train_df.SplitGroup.isnull(),'SplitGroup']='Train'

In [39]:
test_train_df.to_csv("/gpfs/projects/sanlab/bsmith16/data/train_test_markers_"+ts+".csv",index=False)

In [40]:
test_train_df.SplitGroup.value_counts()

Train      60
Holdout    15
Name: SplitGroup, dtype: int64

In [41]:
test_train_df.shape

(75, 2)

# Follow-up October 2021

We now want to mark out the remainder of the subjects, **regardless of whether they ahve a wtp neural dataset**. This is because:

1. we're no longer doing this just for WTP
2. we want to get more data

Note that we want to maintain the previous assignments while also assigning new subjects to the train and holdout group. However we are now assigning subjects to groups of 50% test/train split.

In [214]:
test_train_df_raw = pd.read_csv(ml_data_path + "/train_test_markers_20210601T183243.csv")

In [215]:
test_train_df_raw['AssignedInJune']=True

This time we will classify everything that has a folder in fMRIPrep.

In [216]:
import glob

In [217]:
fmriprep_dir = '/gpfs/projects/sanlab/shared/DEV/bids_data/derivatives/fmriprep'

subject_dirs_all = glob.glob(fmriprep_dir + "/sub-DEV*/")

import re
sub_labels_all = [(re.search("DEV(\\d\\d\\d)",sid)[0]) for sid in subject_dirs_all]
sub_ids_all = [(int)(re.search("DEV(\\d\\d\\d)",sid)[1]) for sid in subject_dirs_all]

sub_labels_all.sort()

In [218]:
test_train_assignments = pd.DataFrame({'sub_label':sub_labels_all})
test_train_assignments = pd.merge(test_train_assignments,test_train_df_raw,how='left') #merge in pre-existing assignments

In [219]:
test_train_assignments.AssignedInJune[test_train_assignments.AssignedInJune.isna()]=False

In [220]:
print(test_train_assignments.SplitGroup.value_counts())
print(test_train_assignments.SplitGroup.isna().value_counts())


Train      60
Holdout    15
Name: SplitGroup, dtype: int64
True     114
False     75
Name: SplitGroup, dtype: int64


In [221]:
#order the test train data so that the pre-assigned values come first.
test_train_assignments.sort_values(['AssignedInJune','sub_label'],ascending=[False,True],inplace=True)
test_train_assignments

,sub_label,SplitGroup,AssignedInJune
0,DEV001,Train,True
1,DEV004,Holdout,True
2,DEV005,Train,True
3,DEV006,Train,True
5,DEV008,Holdout,True
...,...,...,...
184,DEV221,NaN,False
185,DEV222,NaN,False
186,DEV223,NaN,False
187,DEV224,NaN,False


In [222]:
unassigned_count = np.sum(test_train_assignments.SplitGroup.isna())
unassigned_count

114

In [223]:
unassigned_subjects = test_train_assignments.sub_label[test_train_assignments.SplitGroup.isna()]

We want 50% of the TOTAL in train and 50% in test.

In [224]:
total_sample_count = test_train_assignments.shape[0]
total_train = np.floor(total_sample_count/2).astype(int)
total_test = total_sample_count-total_train
remaining_train = total_train - np.sum(test_train_assignments.SplitGroup=='Train')
remaining_test = total_test - np.sum(test_train_assignments.SplitGroup=='Holdout')

In [225]:
print(total_sample_count,total_train,total_test,remaining_train,remaining_test)

189 94 95 34 80


In [226]:
import datetime
#datetime.datetime.now().timestamp()
current_timestamp = 1635381444.165102

In [227]:
ts = datetime.datetime.fromtimestamp(current_timestamp).strftime('%Y%m%dT%H%M%S')

In [228]:
random.seed(a=1635381444.165102)

Now, although we've set a random seed, we can't rely on this to make our analysis deterministic because it's possible the list of folders will change. therefore, the only way to be absolutely confident we don't change sets as more folders are added is to continue to recurse off old assingments and add as necessary.

In [229]:
new_train_subjects = random.sample(set(unassigned_subjects),k=remaining_train)

In [230]:
test_train_assignments.SplitGroup[test_train_assignments.sub_label.isin(new_train_subjects)]='Train'

In [231]:
test_train_assignments.SplitGroup[test_train_assignments.SplitGroup.isna()]='Holdout'

In [232]:
print(test_train_assignments.SplitGroup.value_counts())

Holdout    95
Train      94
Name: SplitGroup, dtype: int64


In [233]:
raise Exception("this codee was only supposed to run once. don't run it again.")
test_train_assignments.to_csv(ml_data_path +"/train_test_markers_"+ts+".csv",index=False)

In [234]:
ml_data_path

'/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml'

In [235]:
test_train_assignments

,sub_label,SplitGroup,AssignedInJune
0,DEV001,Train,True
1,DEV004,Holdout,True
2,DEV005,Train,True
3,DEV006,Train,True
5,DEV008,Holdout,True
...,...,...,...
184,DEV221,Train,False
185,DEV222,Holdout,False
186,DEV223,Holdout,False
187,DEV224,Holdout,False


# Follow-up August 2022

In [177]:
import pandas as pd
import glob
import numpy as np
import random

In [178]:
last_test_train_split = pd.read_csv(
    "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/train_test_markers_20211027T173724.csv"
)
test_train_df_raw = last_test_train_split

In [179]:
last_test_train_split

,sub_label,SplitGroup,AssignedInJune
0,DEV001,Train,True
1,DEV004,Holdout,True
2,DEV005,Train,True
3,DEV006,Train,True
4,DEV008,Holdout,True
...,...,...,...
184,DEV221,Train,False
185,DEV222,Holdout,False
186,DEV223,Holdout,False
187,DEV224,Holdout,False


In [180]:
fmriprep_dir = '/gpfs/projects/sanlab/shared/DEV/bids_data/derivatives/fmriprep'

subject_dirs_all = glob.glob(fmriprep_dir + "/sub-DEV*/")

import re
sub_labels_all = [(re.search("DEV(\\d\\d\\d)",sid)[0]) for sid in subject_dirs_all]
sub_ids_all = [(int)(re.search("DEV(\\d\\d\\d)",sid)[1]) for sid in subject_dirs_all]

sub_labels_all.sort()

In [181]:
test_train_assignments = pd.DataFrame({'sub_label':sub_labels_all})
test_train_assignments = pd.merge(test_train_assignments,test_train_df_raw,how='left') #merge in pre-existing assignments

In [182]:
test_train_assignments["Assigned"] = ""
test_train_assignments["Assigned"][test_train_assignments["AssignedInJune"]==True]="June"
test_train_assignments["Assigned"][test_train_assignments["AssignedInJune"]==False]="October2021"

In [183]:
test_train_assignments['AlreadyAssigned']=test_train_assignments["Assigned"]!=""

In [184]:
test_train_assignments.Assigned.value_counts()

October2021    114
June            75
                58
Name: Assigned, dtype: int64

In [185]:
test_train_assignments.sub_label.sort_values()

0      DEV001
1      DEV004
2      DEV005
3      DEV006
4      DEV007
        ...  
242    DEV285
243    DEV286
244    DEV290
245    DEV291
246    DEV293
Name: sub_label, Length: 247, dtype: object

In [186]:
#order the test train data so that the pre-assigned values come first.
test_train_assignments.sort_values(['AlreadyAssigned','sub_label'],ascending=[False,True],inplace=True)

In [187]:
unassigned_count = np.sum(test_train_assignments.SplitGroup.isna())
unassigned_count

58

In [188]:
unassigned_subjects = test_train_assignments.sub_label[test_train_assignments.SplitGroup.isna()]





In [189]:
total_sample_count = test_train_assignments.shape[0]
total_train = np.floor(total_sample_count/2).astype(int)
total_test = total_sample_count-total_train
remaining_train = total_train - np.sum(test_train_assignments.SplitGroup=='Train')
remaining_test = total_test - np.sum(test_train_assignments.SplitGroup=='Holdout')

In [190]:
print(total_sample_count,total_train,total_test,remaining_train,remaining_test)

247 123 124 29 29


In [191]:
import datetime
#datetime.datetime.now().timestamp()
current_timestamp = 1660858898.370087

In [192]:
ts = datetime.datetime.fromtimestamp(current_timestamp).strftime('%Y%m%dT%H%M%S')

In [193]:
random.seed(a=current_timestamp)

In [194]:
new_train_subjects = random.sample(set(unassigned_subjects),k=remaining_train)
test_train_assignments.loc[test_train_assignments.sub_label.isin(new_train_subjects),"SplitGroup"]='Train'
test_train_assignments.loc[test_train_assignments.SplitGroup.isna(),"SplitGroup"]='Holdout'
print(test_train_assignments.SplitGroup.value_counts())

Holdout    124
Train      123
Name: SplitGroup, dtype: int64


In [195]:
test_train_assignments.loc[test_train_assignments.Assigned=='',"Assigned"]="August2022"

In [196]:
test_train_assignments.drop(columns='AlreadyAssigned',inplace=True)
test_train_assignments.drop(columns='AssignedInJune',inplace=True)

In [197]:
ml_data_path

'/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml'

In [198]:
test_train_assignments

,sub_label,SplitGroup,Assigned
0,DEV001,Train,June
1,DEV004,Holdout,June
2,DEV005,Train,June
3,DEV006,Train,June
4,DEV007,Holdout,October2021
...,...,...,...
242,DEV285,Holdout,August2022
243,DEV286,Holdout,August2022
244,DEV290,Train,August2022
245,DEV291,Train,August2022


In [199]:
test_train_assignments.groupby(['Assigned','SplitGroup']).count()

sub_label
Assigned    SplitGroup           
August2022  Holdout            29
            Train              29
June        Holdout            15
            Train              60
October2021 Holdout            80
            Train              34

In [200]:
test_train_assignments.groupby(['SplitGroup']).count()

,sub_label,Assigned
SplitGroup,,
Holdout,124,124
Train,123,123


Let's also create a 75/25 split because for some applications we don't necessarily need 50% of the subjects in holdout. So we'll assign all the previously trained subjects to train, but grab an extra 50% of the holdout data which will leave 25% for holdout.

In [201]:
test_train_assignments['SplitGroup_75_25'] = test_train_assignments.SplitGroup

In [202]:
holdout_subjs = test_train_assignments.sub_label[test_train_assignments.SplitGroup=="Holdout"]

In [203]:
num_holdouts_to_include_in_75pc_train_group = int(len(holdout_subjs)*0.5)

In [204]:
num_holdouts_to_include_in_75pc_train_group

62

In [205]:
import datetime
#datetime.datetime.now().timestamp()
current_timestamp = 1660861394.946063
random.seed(a=current_timestamp)

new_train_subjects_75pc = random.sample(set(holdout_subjs),k=num_holdouts_to_include_in_75pc_train_group)

In [206]:
test_train_assignments.loc[test_train_assignments.sub_label.isin(new_train_subjects_75pc),
                           "SplitGroup_75_25"]='Train'



In [207]:
test_train_assignments.SplitGroup_75_25.value_counts()

Train      185
Holdout     62
Name: SplitGroup_75_25, dtype: int64

In [208]:
test_train_assignments

,sub_label,SplitGroup,Assigned,SplitGroup_75_25
0,DEV001,Train,June,Train
1,DEV004,Holdout,June,Holdout
2,DEV005,Train,June,Train
3,DEV006,Train,June,Train
4,DEV007,Holdout,October2021,Train
...,...,...,...,...
242,DEV285,Holdout,August2022,Holdout
243,DEV286,Holdout,August2022,Train
244,DEV290,Train,August2022,Train
245,DEV291,Train,August2022,Train


In [209]:
62/(185+62)

0.25101214574898784

In [211]:
raise Exception("this codee was only supposed to run once. don't run it again.")
test_train_assignments.to_csv(ml_data_path +"/train_test_markers_"+ts+".csv",index=False)

Exception: this codee was only supposed to run once. don't run it again.

The 80/20 split is a bit small, less than 40 subjects once you do data cleaning. Let's do a 75/25 split too.